# Calculating the reserve Factors of the panels 

# Please enter your name 

In [ ]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *
 

In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

# Constants (Changed based on your data)

In [4]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
nu = personal_data[2]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}, nu = {nu}. Please verify.")

#Panel constants 
length = 750
width = 200


Your personal data is: sigma_yield = 490, EModulus = 65143.57, nu = 0.34. Please verify.


# Open stress values & properties of panels 

In [5]:
#Imoport panel stress file and reformat (OLD VERSION)
# paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
# paneldf = paneldf.drop(['sigmaZZ', 'sigmaXZ', 'sigmaYZ'],axis=1)
# paneldf = paneldf.reset_index()


########
# New version of importing panel stress file
########

paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = add_component_names_to_elements(paneldf_v2, element_component_df)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)
paneldf_v2.reset_index(drop=True, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True) # @yannisendr this is possible, since pandas allows non-unique indices, however operations that require an unique index won't work 

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

#Import panel properties and reformat
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})

paneldf = paneldf_v2

#print(paneldf.head(5))
print(paneldf_v2.head(5))
#print(panelPropertiesdf.head(5))

                  Load Case  sigmaXX  sigmaXY  sigmaYY Component Name
Element ID                                                           
1           Subcase 1 (LC1)   -65.34    25.72    12.50         panel1
2           Subcase 1 (LC1)   -81.32    26.56    20.46         panel1
3           Subcase 1 (LC1)   -66.28    36.24    20.33         panel1
4           Subcase 1 (LC1)   -95.73    33.83     2.37         panel2
5           Subcase 1 (LC1)   -87.72    26.51    24.84         panel2


# Split up the loadcases 

In [6]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name_x,Component Name_y,thickness,mass
Element ID,,,,,,,,
1,Subcase 1 (LC1),-65.34,25.72,12.50,panel1,panel1,4,0.00054
2,Subcase 1 (LC1),-81.32,26.56,20.46,panel1,panel1,4,0.00054
3,Subcase 1 (LC1),-66.28,36.24,20.33,panel1,panel1,4,0.00054
4,Subcase 1 (LC1),-95.73,33.83,2.37,panel2,panel2,4,0.00054
5,Subcase 1 (LC1),-87.72,26.51,24.84,panel2,panel2,4,0.00054


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [7]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC1['length'] = [length]*len(panelLC1)
panelLC1['width'] = [width]*len(panelLC1)
panelLC1

#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC2['length'] = [length]*len(panelLC2)
panelLC2['width'] = [width]*len(panelLC2)
panelLC2

#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC3['length'] = [length]*len(panelLC3)
panelLC3['width'] = [width]*len(panelLC3)
panelLC3

panelLC1

KeyError: 'Component Name'

# Now we can apply the functions to the load cases 

In [ ]:
panelLC1[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,,,,
panel1,-71.460000,17.88,29.706667,4.0,750,200,5.624444,5.051199,1.019403
panel10,-71.460000,17.88,29.706667,4.0,750,200,5.624444,5.051199,1.019403
panel2,-93.236667,17.88,29.606667,4.0,750,200,5.624444,4.830927,0.769982
panel3,-93.370000,17.88,29.613333,4.0,750,200,5.624444,4.829528,0.768739
panel4,-92.716667,17.88,30.066667,4.0,750,200,5.624444,4.836425,0.772770
panel5,-92.223333,17.88,52.546667,4.0,750,200,5.624444,4.841711,0.661935
panel6,-92.223333,17.88,52.546667,4.0,750,200,5.624444,4.841711,0.661935
panel7,-92.716667,17.88,30.066667,4.0,750,200,5.624444,4.836425,0.772770
panel8,-93.370000,17.88,29.613333,4.0,750,200,5.624444,4.829528,0.768739


# Combining back into 1 Data Frame to output 

In [ ]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length','thickness', 'width'], axis=1)
panelLC2 = panelLC2.drop(['length','thickness', 'width'], axis=1)
panelLC3 = panelLC3.drop(['length','thickness', 'width'], axis=1)
panelLC1.head(3)


,sigmaXX,sigmaYY,sigmaXY,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,
panel1,-71.460000,17.88,29.706667,5.624444,5.051199,1.019403
panel10,-71.460000,17.88,29.706667,5.624444,5.051199,1.019403
panel2,-93.236667,17.88,29.606667,5.624444,4.830927,0.769982


In [ ]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'k_shear':'k_shearLC1', 'k_biaxial':'k_biaxialLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'k_shear':'k_shearLC2', 'k_biaxial':'k_biaxialLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'k_shear':'k_shearLC3', 'k_biaxial':'k_biaxialLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF
Component Name,,,,,,
panel1,-71.460000,17.88,29.706667,5.624444,5.051199,1.019403
panel10,-71.460000,17.88,29.706667,5.624444,5.051199,1.019403
panel2,-93.236667,17.88,29.606667,5.624444,4.830927,0.769982
panel3,-93.370000,17.88,29.613333,5.624444,4.829528,0.768739
panel4,-92.716667,17.88,30.066667,5.624444,4.836425,0.772770
panel5,-92.223333,17.88,52.546667,5.624444,4.841711,0.661935
panel6,-92.223333,17.88,52.546667,5.624444,4.841711,0.661935
panel7,-92.716667,17.88,30.066667,5.624444,4.836425,0.772770
panel8,-93.370000,17.88,29.613333,5.624444,4.829528,0.768739


In [ ]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-71.460000,17.88,29.706667,5.624444,5.051199,1.019403,-73.090000,-17.88,29.706667,5.624444,3.040367,0.627979,-23.740000,5.94,118.423333,5.624444,5.051201,0.504025
panel10,-71.460000,17.88,29.706667,5.624444,5.051199,1.019403,-73.090000,-17.88,29.706667,5.624444,3.040367,0.627979,-23.740000,5.94,118.423333,5.624444,5.051201,0.504025
panel2,-93.236667,17.88,29.606667,5.624444,4.830927,0.769982,-92.500000,-17.88,29.610000,5.624444,3.227660,0.532712,-30.973333,5.94,118.043333,5.624444,4.830966,0.481009
panel3,-93.370000,17.88,29.613333,5.624444,4.829528,0.768739,-93.896667,-17.88,29.610000,5.624444,3.238835,0.526946,-31.016667,5.94,118.050000,5.624444,4.829599,0.480811
panel4,-92.716667,17.88,30.066667,5.624444,4.836425,0.772770,-93.710000,-17.88,30.066667,5.624444,3.237356,0.526795,-30.803333,5.94,119.853333,5.624444,4.836377,0.469848
panel5,-92.223333,17.88,52.546667,5.624444,4.841711,0.661935,-93.256667,-17.88,52.546667,5.624444,3.233746,0.472618,-30.636667,5.94,209.480000,5.624444,4.841753,0.175739
panel6,-92.223333,17.88,52.546667,5.624444,4.841711,0.661935,-93.256667,-17.88,52.546667,5.624444,3.233746,0.472618,-30.636667,5.94,209.480000,5.624444,4.841753,0.175739
panel7,-92.716667,17.88,30.066667,5.624444,4.836425,0.772770,-93.710000,-17.88,30.066667,5.624444,3.237356,0.526795,-30.803333,5.94,119.853333,5.624444,4.836377,0.469848
panel8,-93.370000,17.88,29.613333,5.624444,4.829528,0.768739,-93.896667,-17.88,29.610000,5.624444,3.238835,0.526946,-31.016667,5.94,118.050000,5.624444,4.829599,0.480811


# ROUND & Output the file 

In [ ]:
outputDf = outputDf.round(rounding_digits)
outputDf.to_excel(f'../data/{name}/output/processed_e.xlsx')
outputDf.head(10)

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-71.46,17.88,29.71,5.62,5.05,1.02,-73.09,-17.88,29.71,5.62,3.04,0.63,-23.74,5.94,118.42,5.62,5.05,0.50
panel10,-71.46,17.88,29.71,5.62,5.05,1.02,-73.09,-17.88,29.71,5.62,3.04,0.63,-23.74,5.94,118.42,5.62,5.05,0.50
panel2,-93.24,17.88,29.61,5.62,4.83,0.77,-92.50,-17.88,29.61,5.62,3.23,0.53,-30.97,5.94,118.04,5.62,4.83,0.48
panel3,-93.37,17.88,29.61,5.62,4.83,0.77,-93.90,-17.88,29.61,5.62,3.24,0.53,-31.02,5.94,118.05,5.62,4.83,0.48
panel4,-92.72,17.88,30.07,5.62,4.84,0.77,-93.71,-17.88,30.07,5.62,3.24,0.53,-30.80,5.94,119.85,5.62,4.84,0.47
panel5,-92.22,17.88,52.55,5.62,4.84,0.66,-93.26,-17.88,52.55,5.62,3.23,0.47,-30.64,5.94,209.48,5.62,4.84,0.18
panel6,-92.22,17.88,52.55,5.62,4.84,0.66,-93.26,-17.88,52.55,5.62,3.23,0.47,-30.64,5.94,209.48,5.62,4.84,0.18
panel7,-92.72,17.88,30.07,5.62,4.84,0.77,-93.71,-17.88,30.07,5.62,3.24,0.53,-30.80,5.94,119.85,5.62,4.84,0.47
panel8,-93.37,17.88,29.61,5.62,4.83,0.77,-93.90,-17.88,29.61,5.62,3.24,0.53,-31.02,5.94,118.05,5.62,4.83,0.48
